In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

from WindPy import *
w.start()

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2020 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


.ErrorCode=0
.Data=[OK!]

## 一、数据准备和预处理

### 名义本金数据采用2020年8月31日的数据

In [2]:
short_term_amount= 400000000
long_term_amount= 40000000

### 中债1年和10年收益率数据提取

In [3]:
# bond_yeild_da=w.edb("S0059744,S0059749", "2014-09-22", "2020-09-29","Fill=Previous")

# bond_yeild_dat=pd.DataFrame(bond_yeild_da.Data).T

# bond_yeild_dat.columns=['1yearyeild','10yearyeild']

# bond_yeild_dat.index=bond_yeild_da.Times

# bond_yeild_dat['Date']=bond_yeild_dat.index
# bond_yeild_dat['Date']=bond_yeild_dat.Date.apply(lambda x: x.strftime('%Y-%m-%d'))

# bond_yeild_dat.to_csv('bond_yeild_dat.csv')

In [84]:
bond_yeild_dat = pd.read_csv('bond_yeild_dat.csv',index_col=0)

In [95]:
bond_yeild_dat.index[0]

'2014-09-22'

### TS数据提取

In [115]:
ts_dat = w.wsi("TS.CFE", "close", "2018-03-23 09:15:00", "2018-08-20 15:15:00", "BarSize=5")
ts_data = pd.DataFrame(ts_dat.Data).T
ts_data.columns=['ts_close']
ts_data.index=ts_dat.Times

In [116]:
ts_data

,ts_close
2018-08-17 09:20:00,99.320
2018-08-17 09:25:00,99.340
2018-08-17 09:30:00,99.375
2018-08-17 09:35:00,99.360
2018-08-17 09:40:00,99.330
...,...
2018-08-20 14:55:00,99.210
2018-08-20 15:00:00,99.210
2018-08-20 15:05:00,99.200
2018-08-20 15:10:00,99.210


In [86]:
# ts_data.to_csv('ts_data.csv')

In [90]:
ts_data = pd.read_csv('ts_data.csv',index_col=0)

In [91]:
ts_data.head(3)

,ts_close
2018-08-17 09:20:00,99.320
2018-08-17 09:25:00,99.340
2018-08-17 09:30:00,99.375


In [98]:
ts_data['Datetime']=ts_data.index

In [105]:
ts_data['Datetime']=ts_data.Datetime.apply(lambda x:datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))

### 期货数据和信号数据提取

In [4]:
future_signal_data= pd.read_excel('./T_5MinutesData.xlsx',sheet_name='Sheet2',index_col=0)

# bond_yeild_data= pd.read_excel('./T_5MinutesData.xlsx',sheet_name=r'国债收益率')

future_signal_data.rename(columns={'Unnamed: 1':'future_close'},inplace=True)

In [5]:
# bond_yeild_data = bond_yeild_data.iloc[2:] 

# bond_yeild_data.rename(columns={'Wind':'Date','Unnamed: 1':'yeild_close'},inplace=True)

# bond_yeild_data.reset_index(drop=True,inplace=True)

# bond_yeild_data.Date[0]

# bond_yeild_data['Date']=bond_yeild_data.Date.apply(lambda x: x.strftime('%Y-%m-%d'))

In [6]:
future_data= future_signal_data[['future_close','position','pnl']]

future_data['Datetime'] =  future_data.index

future_data['Date']= [ date.strftime('%Y-%m-%d')  for date in future_data.index]

data_set=pd.merge(future_data,bond_yeild_dat,on=['Date'],how='inner')

In [106]:
data_sete=pd.merge(data_set,ts_data,on=['Datetime'],how='inner')

### 整合成为一个数据集

In [108]:
data_sete.fillna(0,inplace=True)

data_set_all=data_sete[['Datetime','future_close','ts_close','1yearyeild','10yearyeild','position','pnl']]

In [109]:
data_set_all.head(5)

,Datetime,future_close,ts_close,1yearyeild,10yearyeild,position,pnl
0,2018-08-17 09:20:00,94.745,99.320,2.8833,3.6524,-1.0,0.0
1,2018-08-17 09:25:00,94.705,99.340,2.8833,3.6524,-1.0,0.0
2,2018-08-17 09:30:00,94.700,99.375,2.8833,3.6524,-1.0,0.0
3,2018-08-17 09:35:00,94.685,99.360,2.8833,3.6524,-1.0,0.0
4,2018-08-17 09:40:00,94.625,99.330,2.8833,3.6524,-1.0,0.0


In [110]:
len(data_set_all)

26274

## 二、组合久期和对冲比例计算

In [9]:
data_set_all['bond_ret']=str(0)
data_set_all['future_ret']=str(0)
data_set_all['bond_pnl']=str(0)
data_set_all['future_pnl']=str(0)

### 短债久期选择通过以下5只加权计算而得 ，长债久期选择20国债06（8.4856）,名义本金加权算得组合久期

In [10]:
duration_1504  = 0.0629
duration_1903  = 1.3750
duration_1909 =0.7707
duration_2002  =1.3538 
duration_2010 = 0.7813


In [59]:
short_term_duration=duration_1504*2/(2+5+10+18+15) + duration_1903*5/(short_term_amount/10000000) + duration_1909*10/(short_term_amount/10000000) + duration_2002*8/(short_term_amount/10000000) + duration_2010*15/(short_term_amount/10000000)

In [60]:
long_term_duration=8.4856

In [61]:
port_duration= short_term_duration*(short_term_amount/(short_term_amount+long_term_amount))+long_term_duration*(long_term_amount/(short_term_amount+long_term_amount))

In [62]:
port_duration

1.3185203703703703

### CTD久期选择20抗疫国债04，久期（8.5489）

In [63]:
ctd_duration=8.5489

### 利用组合久期计算期货对冲数，选用的组合价格为中债国债全价指数。
### 在开仓期初计算需要对冲的期货手数、组合收益率
### 期末再计算一次组合收益率，组合收益率差值计算债券相对和绝对收益
### 期初对冲手数计算期货 绝对收益

In [64]:
for i in range(len(data_set_all)-1):
    if (data_set_all['position'][i]==0)&(data_set_all['position'][i+1]==-1):
        
        last_future_price=data_set_all['future_close'][i]
        
        
        #组合ytm:久期和名义额加权，有缺陷，实际上需要久期和市值加权
        last_portfolio_yeild= (data_set_all['1yearyeild'][i]*short_term_duration*short_term_amount\
        +data_set_all['10yearyeild'][i]*long_term_duration*long_term_amount)/(short_term_duration*short_term_amount+long_term_duration*long_term_amount)
        print(last_portfolio_yeild)
        #对冲手数：组合久期*名义额/（合约乘数*ctd_久期）
        hedge_amount=port_duration*(short_term_amount+long_term_amount)/(ctd_duration*1000000)
        print(hedge_amount)
        
    elif(data_set_all['position'][i]==-1)&(data_set_all['position'][i+1]==0):
        
        current_portfolio_yeild=(data_set_all['1yearyeild'][i]*short_term_duration*short_term_amount\
        +data_set_all['10yearyeild'][i]*long_term_duration*long_term_amount)/(short_term_duration*short_term_amount+long_term_duration*long_term_amount)
#         print(current_portfolio_yeild)
        #债券收益率：组合收益率变动乘上组合久期
        data_set_all['bond_ret'][i]=str(np.round(-(current_portfolio_yeild-last_portfolio_yeild)*port_duration,4))
        data_set_all['future_ret'][i]=str(np.round(-1*(data_set_all['future_close'][i]/last_future_price-1),4))
        #债券绝对收益：债券收益率*开仓时记录的指数价格/100*名义额
        data_set_all['bond_pnl'][i]=str(np.round(-((current_portfolio_yeild-last_portfolio_yeild)/100)*port_duration*(short_term_amount+long_term_amount),4))
        #期货绝对收益：期初开仓手数*合约乘数*期货价格变动/100
        data_set_all['future_pnl'][i]=str(np.round((data_set_all['future_close'][i]-last_future_price)*hedge_amount*1000000/100*-1,4))
       
        

3.2847297224301655
83.28568587771527
3.3369374833743217
83.28568587771527
3.3747501221206146
83.28568587771527
3.4002773942733224
83.28568587771527
3.364414201805896
83.28568587771527
3.4250425864570406
83.28568587771527
3.415006001676964
83.28568587771527
3.3642210780602837
83.28568587771527
3.305109548020298
83.28568587771527
3.262727726506002
83.28568587771527
3.201120188314342
83.28568587771527
3.1744448038696578
83.28568587771527
3.0906066674063664
83.28568587771527
3.0398483552691635
83.28568587771527
3.064585386818277
83.28568587771527
3.010177624610078
83.28568587771527
2.9653006884821793
83.28568587771527
2.844145704991987
83.28568587771527
2.8683093417003627
83.28568587771527
2.9106222006710665
83.28568587771527
2.8110159977303404
83.28568587771527
2.650039291939197
83.28568587771527
2.7284794255906943
83.28568587771527
2.738766342603451
83.28568587771527
2.6964408461504967
83.28568587771527
2.626157699497613
83.28568587771527
2.620546173249757
83.28568587771527
2.61112355993

3.571576056494303
83.28568587771527
3.6536607535663572
83.28568587771527
3.5954940122275105
83.28568587771527
3.5756509966980388
83.28568587771527
3.5198631912033833
83.28568587771527
3.5545252774925875
83.28568587771527
3.49670731796725
83.28568587771527
3.4920654124081283
83.28568587771527
3.511628606139598
83.28568587771527
3.5205221766542465
83.28568587771527
3.517823506849007
83.28568587771527
3.505797120650111
83.28568587771527
3.4975561010448017
83.28568587771527
3.512601999716292
83.28568587771527
3.4896512245067073
83.28568587771527
3.4875006707855745
83.28568587771527
3.4880687423191827
83.28568587771527
3.5082334877338655
83.28568587771527
3.471360983903112
83.28568587771527
3.500792469392599
83.28568587771527
3.4843354849220725
83.28568587771527
3.4530454620148006
83.28568587771527
3.4607583222495473
83.28568587771527
3.460198011660096
83.28568587771527
3.4688381453115937
83.28568587771527
3.490524397858992
83.28568587771527
3.514841248678021
83.28568587771527
3.57732638999

2.8281079988651703
83.28568587771527
2.8338738517221183
83.28568587771527
2.8499935869472086
83.28568587771527
2.8629840528313864
83.28568587771527
2.905190703805191
83.28568587771527
2.946720195898602
83.28568587771527
2.973361215503911
83.28568587771527
2.965732611892399
83.28568587771527
2.9465545620020195
83.28568587771527
2.9527980217724417
83.28568587771527
2.9435951386304233
83.28568587771527
2.9552933641947132
83.28568587771527
2.894211766696957
83.28568587771527
2.904718640423258
83.28568587771527
2.891899349439116
83.28568587771527
2.8930221880306344
83.28568587771527
2.8908940286600724
83.28568587771527
2.884895803095782
83.28568587771527
2.9083838300788907
83.28568587771527
2.891035272281921
83.28568587771527
2.7365295110540577
83.28568587771527
2.765196251128861
83.28568587771527
2.7660037893205205
83.28568587771527
2.692344146988557
83.28568587771527
2.6815596676128264
83.28568587771527
2.662866098362221
83.28568587771527
2.5848758552305404
83.28568587771527
2.44000556375

In [65]:
data_set_all['bond_pnl']=data_set_all.bond_pnl.astype('float')
data_set_all['bond_ret']=data_set_all.bond_ret.astype('float')
data_set_all['future_pnl']=data_set_all.future_pnl.astype('float')
data_set_all['future_ret']=data_set_all.future_ret.astype('float')

In [66]:
data_set_all.rename(columns={'pnl':'future_return'},inplace=True)

In [67]:
data_set_all[data_set_all['future_return']!=0]

,Datetime,future_close,1yearyeild,10yearyeild,position,future_return,bond_ret,future_ret,bond_pnl,future_pnl
112,2015-03-25 09:15:00,97.020,3.1183,3.5037,-1.0,-0.0001,-0.0228,-0.0001,-123158.808,-8328.5686
229,2015-03-27 09:40:00,96.160,3.1833,3.5678,-1.0,0.0070,-0.0391,0.0070,-211184.862,562178.3797
336,2015-03-31 09:15:00,95.770,3.2236,3.6483,-1.0,0.0050,-0.0677,0.0050,-365343.347,399771.2922
448,2015-04-02 09:15:00,96.300,3.1743,3.5438,-1.0,-0.0065,0.0657,-0.0065,354789.627,-516371.2524
562,2015-04-07 09:25:00,95.960,3.1801,3.5987,-1.0,0.0037,-0.0201,0.0037,-108509.904,299828.4692
...,...,...,...,...,...,...,...,...,...,...
73231,2020-08-07 13:00:00,98.200,2.2225,2.9918,-1.0,0.0019,0.0026,0.0019,14186.339,154078.5189
73354,2020-08-11 14:25:00,98.475,2.2213,2.9544,-1.0,0.0008,-0.0147,-0.0028,-79325.748,-229035.6362
73424,2020-08-13 10:00:00,98.585,2.2522,2.9555,-1.0,-0.0011,-0.0033,-0.0019,-17803.179,-158242.8032
73524,2020-08-17 09:30:00,98.930,2.2650,2.9421,-1.0,-0.0020,0.0032,-0.0020,17372.190,-166571.3718


In [68]:
data_set_all['future_return']=data_set_all.future_return.apply(lambda x : round(x,4))

In [69]:
len(data_set_all[data_set_all['future_return']!=0])

552

In [70]:
data_set_all[data_set_all['future_return']!=data_set_all['future_ret']][['Datetime','future_return','future_ret']]

,Datetime,future_return,future_ret
1012,2015-04-17 09:35:00,-0.0005,-0.0020
2339,2015-05-21 14:15:00,0.0018,-0.0005
2808,2015-06-03 09:55:00,-0.0006,-0.0034
3870,2015-07-01 09:45:00,-0.0012,-0.0018
4538,2015-07-17 09:25:00,-0.0013,-0.0025
...,...,...,...
72417,2020-07-17 10:00:00,-0.0003,-0.0027
72628,2020-07-23 09:55:00,-0.0002,-0.0031
72810,2020-07-28 13:15:00,0.0019,0.0001
73354,2020-08-11 14:25:00,0.0008,-0.0028


In [71]:
results=data_set_all[data_set_all['future_return']!=0]

In [72]:
results.to_csv('./hegde_results.csv')

In [73]:
# future_total_pnl = data_set_all.future_return.apply(lambda x : x+1).cumprod()

### 期货收益率

In [74]:
# "{:.4f}".format(future_total_pnl.values[-1]-1)

In [75]:
future_total_pnl = results.future_ret.apply(lambda x : x+1).cumprod()

In [76]:
"{:.4f}".format(future_total_pnl.values[-1]-1)

'0.1482'

In [77]:
yeild_total_pnl =results.bond_ret.apply(lambda x : x/100+1).cumprod()

### 国债收益率

In [78]:
"{:.4f}".format(yeild_total_pnl.values[-1]-1)

'-0.0123'

### 累计收益率相关性

In [79]:
yeild_total_pnl.corr(future_total_pnl)

-0.9506763805806945

### 累计绝对收益相关性

In [80]:
results.bond_pnl.cumsum().corr(results.future_pnl.cumsum())

-0.9504721005059956

### 对冲期间债券绝对收益

In [81]:
results.bond_pnl.sum()

-6687482.716

### 期货绝对收益

In [82]:
results.future_pnl.sum()

11322688.9957

### 对冲期间组合绝对收益

In [36]:
results.bond_pnl.sum()+results.future_pnl.sum()

6008489.327199999